# 参数设定

In [1]:
import os
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

Fs = 10 # 采样频率
interval = 10  # 时程10分钟
interval_gust = 3  # 阵风持续时间3秒
start_time_main = '2011-08-06 13:00'
end_time_main = '2011-08-07 05:10' # 数据过饱和十分钟，画图时只用到5:00

original_data_path = 'E:/【论文】/【小论文】/航博/航博数据/Wind/TJHBG-E.csv' # 原始数据
pkl_path = 'E:/【论文】/【小论文】/航博/航博数据/Wind/Wind(East)(AMD)/' # pkl保存位置
if not os.path.exists(pkl_path): os.makedirs(pkl_path)

# 如果已经存在会话，就不必新开一个
import matlab.engine
engine_list = matlab.engine.find_matlab()
if len(engine_list):
    try:
        engine = matlab.engine.connect_matlab(engine_list[-1])
    except:
        engine = matlab.engine.start_matlab()
else:
    engine = matlab.engine.start_matlab()

# 计算

In [2]:
# 读取文件
with open(original_data_path) as f:
    data = pd.read_csv(f, index_col='Date', parse_dates=True) # 将Date作索引，并且使日期可解析

# 更正日期索引并增加数字时间项'time2number'
date = data.index; temp, new_date = date[0], [date[0]]
for ii in range(1,len(data)):
    if date[ii] == date[ii-1]:
        temp += pd.Timedelta('0.1 seconds')
    else:
        temp = date[ii]
    new_date.append(temp)
data.index = new_date

# 框定时间范围
start_time = np.datetime64(start_time_main); end_time = np.datetime64(end_time_main)
data = data[start_time: end_time]
data['time2number'] = pd.to_numeric(data.index)/100000000
data['time2number'] = data['time2number'] - data['time2number'][0]

# 计算实时风速 ux,uy,uz,uxy,uxyz
data.rename(columns = {'A.3D Wind Speed u':'uxyz'}, inplace = True); data = data[['time2number', 'uxyz', 'Offset','Schedule','A.Azimuth','A.Elevation','A.Sonic Temp']]
data['uxy'] = data['uxyz'] * np.cos(np.deg2rad(data['A.Elevation']))
data['ux'] = data['uxyz'] * np.cos(np.deg2rad(data['A.Elevation'])) * np.sin(np.deg2rad(data['A.Azimuth']))
data['uy'] = data['uxyz'] * np.cos(np.deg2rad(data['A.Elevation'])) * np.cos(np.deg2rad(data['A.Azimuth']))
data['uz'] = data['uxyz'] * np.sin(np.deg2rad(data['A.Elevation']))
data = data.drop(['Offset','Schedule','A.Azimuth','A.Elevation','A.Sonic Temp'], axis=1) # 删除不需要的数据

# 计算时变平均风速 ux_mean,uy_mean,uz_mean,uxy_mean,uxyz_mean（AMD_trend）
time_series = data['time2number']
w = float(5000/len(time_series)); # 1500或5000
Fs = float(Fs); nbsym1 = float(10000); nbsym2 = float(10000); if_draw = float(0)
engine.cd(r'E:\【论文】\【小论文】\模态识别\Matlab脚本\Method_Functions\Regression')
for ii in ['uxyz', 'uxy', 'ux', 'uy', 'uz']:
    speed = data[ii]
    IMF = engine.AMD_trend_py(matlab.double(speed.tolist())[0], Fs, w, nbsym1, nbsym2, ii, if_draw)
    exec('data["'+ii+'_mean"] = list(IMF[0])')

# 计算水平时变平均风向角Angle、竖向时变平均风向角Angle_z
cos_theta = (data['uy_mean'] / data['uxy_mean'])
sin_theta = (data['ux_mean'] / data['uxy_mean'])
Angle = np.zeros(len(time_series))
Angle[cos_theta>=0] = np.rad2deg(np.arcsin(sin_theta[cos_theta>=0]))
Angle[cos_theta<0] = 180 - np.rad2deg(np.arcsin(sin_theta[cos_theta<0]))
Angle[Angle<0] = Angle[Angle<0] + 360
Angle_z = np.rad2deg(np.arcsin(data['uz_mean'] / data['uxyz_mean']))
data['Angle'] = Angle; data['Angle_z'] = Angle_z

# 计算顺风向脉动风速u，横风向脉动风速v，竖向脉动风速w
data['u'] = data['ux'] * np.sin(np.deg2rad(data['Angle'])) + data['uy'] * np.cos(np.deg2rad(data['Angle'])) - data['uxy_mean']
data['v'] = data['ux'] * np.cos(np.deg2rad(data['Angle'])) - data['uy'] * np.sin(np.deg2rad(data['Angle']))
data['w'] = data['uz'] - data['uz_mean']

# 计算湍流度 Turbulence intensity
turbulence_intensity = pd.DataFrame([])
uxy_mean_temp = data['uxy_mean'].resample('%dT'%interval).mean()
turbulence_intensity['turbulence_intensity_u'] = data['u'].resample('%dT'%interval).std()/uxy_mean_temp
turbulence_intensity['turbulence_intensity_v'] = data['v'].resample('%dT'%interval).std()/uxy_mean_temp
turbulence_intensity['turbulence_intensity_w'] = data['w'].resample('%dT'%interval).std()/uxy_mean_temp

# 计算偏度系数 Skewness
Skewness = pd.DataFrame([])
Skewness['Skewness_u'] = data['u'].resample('%dT'%interval).apply(lambda x: stats.skew(x))
Skewness['Skewness_v'] = data['v'].resample('%dT'%interval).apply(lambda x: stats.skew(x))
Skewness['Skewness_w'] = data['w'].resample('%dT'%interval).apply(lambda x: stats.skew(x))

# 计算峰度系数 Kurtosis
Kurtosis = pd.DataFrame([])
Kurtosis['Kurtosis_u'] = data['u'].resample('%dT'%interval).apply(lambda x: stats.kurtosis(x,fisher=False))
Kurtosis['Kurtosis_v'] = data['v'].resample('%dT'%interval).apply(lambda x: stats.kurtosis(x,fisher=False))
Kurtosis['Kurtosis_w'] = data['w'].resample('%dT'%interval).apply(lambda x: stats.kurtosis(x,fisher=False))

# 计算阵风因子 Gust factor
gust_factor = pd.DataFrame([])
gust_factor['gust_factor_u'] = 1 + data['u'].resample('%dT'%interval).apply(lambda x: x.resample('%dS'%interval_gust).mean().max())/data['uxy_mean'].resample('%dT'%interval).mean()
gust_factor['gust_factor_v'] = data['v'].resample('%dT'%interval).apply(lambda x: x.resample('%dS'%interval_gust).mean().max())/data['uxy_mean'].resample('%dT'%interval).mean()
gust_factor['gust_factor_w'] = data['w'].resample('%dT'%interval).apply(lambda x: x.resample('%dS'%interval_gust).mean().max())/data['uxy_mean'].resample('%dT'%interval).mean()

# 计算湍流积分尺度 Turbulence integral scale
engine.cd(r'E:\【论文】\【小论文】\模态识别\Matlab脚本\Method_Functions\Wind')
def calculate_turbulence_integral_scale(x,Fs,U,method='DirectInt'):
    turbulence_integral_scale = engine.turbulence_integral_scale_py(matlab.double(x.tolist())[0],float(Fs),float(U),method)
    return turbulence_integral_scale
turbulence_integral_scale = pd.DataFrame([])
temp = data[['u','v','w','uxy_mean']]
turbulence_integral_scale['turbulence_integral_scale_u'] = temp.resample('%dT'%interval).apply(lambda x : calculate_turbulence_integral_scale(x['u'],Fs,x['uxy_mean'].mean()))
turbulence_integral_scale['turbulence_integral_scale_v'] = temp.resample('%dT'%interval).apply(lambda x : calculate_turbulence_integral_scale(x['v'],Fs,x['uxy_mean'].mean()))
turbulence_integral_scale['turbulence_integral_scale_w'] = temp.resample('%dT'%interval).apply(lambda x : calculate_turbulence_integral_scale(x['w'],Fs,x['uxy_mean'].mean()))

# 保存

In [15]:
columns = pd.DataFrame({'turbulence_intensity':turbulence_intensity.keys(),'gust_factor':gust_factor.keys(),
          'turbulence_integral_scale':turbulence_integral_scale.keys(),'Skewness':Skewness.keys(),'Kurtosis':Kurtosis.keys()})
columns = pd.MultiIndex.from_frame(columns.melt(var_name='type', value_name='direction'))
data1 = pd.concat([turbulence_intensity,gust_factor,turbulence_integral_scale,Skewness,Kurtosis],axis=1)
data1.columns = columns
data0 = data

import pickle
with open(pkl_path+'data0.pkl','wb') as f:
    pickle.dump(data0, f)
with open(pkl_path+'data1.pkl','wb') as f:
    pickle.dump(data1, f)